<a href="https://colab.research.google.com/github/detektor777/colab_list_image/blob/main/enhance_black.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title ##**Upload images** { display-mode: "form" }

import shutil
from tqdm import tqdm
import os
import shutil, sys
import re
import io
import IPython.display
import numpy as np
import PIL.Image
from google.colab import files
import shutil

upload_folder = "/content/upload"
result_folder = "/content/results"

if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)
os.makedirs(upload_folder)

if os.path.isdir(result_folder):
    shutil.rmtree(result_folder)
os.makedirs(result_folder)

basepath = os.getcwd()
uploaded = files.upload()
for filename in uploaded.keys():
    shutil.move(os.path.join(basepath, filename), os.path.join(upload_folder, filename))

In [ ]:
#@title ##**Enhance Blacks** { display-mode: "form" }
import cv2
import numpy as np
import os
from tqdm import tqdm

upload_folder = "/content/upload"
result_folder = "/content/results"

if os.path.isdir(result_folder):
    shutil.rmtree(result_folder)
os.makedirs(result_folder)

def enhance_blacks(image, black_level=0.5, threshold=0.1, transition=0.05, curve_strength=0.5):
    """
    image: входное изображение в формате BGR (OpenCV)
    black_level: сила усиления черных (0.0 - 1.0)
    threshold: порог, ниже которого усиливаются темные тона (0.0 - 1.0)
    transition: ширина зоны плавного перехода (0.0 - 1.0)
    curve_strength: сила нелинейности (0.0 - линейно, 1.0 - сильно затемняет темные)
    """
    image_float = image.astype(np.float32) / 255.0
    corrected = np.copy(image_float)
    mask = image_float < threshold
    linear_factor = (1 - black_level)
    nonlinear_factor = np.power(image_float[mask] / threshold, black_level * curve_strength)
    corrected[mask] = image_float[mask] * (
        (1 - curve_strength) * linear_factor + curve_strength * nonlinear_factor
    )
    transition_start = threshold - transition
    transition_end = threshold
    transition_mask = (image_float >= transition_start) & (image_float < transition_end)
    for channel in range(3):
        alpha = (image_float[:, :, channel] - transition_start) / transition
        corrected[:, :, channel][transition_mask[:, :, channel]] = (
            (1 - alpha) * (
                image_float[:, :, channel] * (
                    (1 - curve_strength) * linear_factor +
                    curve_strength * np.power(image_float[:, :, channel] / threshold, black_level * curve_strength)
                )
            ) +
            alpha * image_float[:, :, channel]
        )[transition_mask[:, :, channel]]
    corrected = np.clip(corrected, 0, 1) * 255
    return corrected.astype(np.uint8)

black_level = 0.9 #@param {type:"slider", min:0.0, max:1.0, step:0.1}
threshold = 0.2 #@param {type:"slider", min:0.0, max:1.0, step:0.05}
transition = 0.1 #@param {type:"slider", min:0.0, max:0.2, step:0.01}
curve_strength = 0.9 #@param {type:"slider", min:0.0, max:1.0, step:0.1}

filenames = os.listdir(upload_folder)
for filename in tqdm(filenames, desc="Processing images"):
    image_path = os.path.join(upload_folder, filename)
    image = cv2.imread(image_path)
    if image is None:
        print(f"Не удалось загрузить изображение: {image_path}")
        continue

    enhanced_image = enhance_blacks(image, black_level, threshold, transition, curve_strength)

    output_path = os.path.join(result_folder, filename)
    cv2.imwrite(output_path, enhanced_image)

print(f"Обработка завершена. Результаты сохранены в {result_folder}")

In [ ]:
#@title ##**Visualize** { display-mode: "form" }
import PIL.Image
import numpy as np
from IPython.display import display
import os

# Папки с исходными и обработанными изображениями
upload_folder = "/content/upload"
result_folder = "/content/results"

# Получение и сортировка списка файлов
filenames_upload = os.listdir(upload_folder)
filenames_upload.sort()
filenames_result = os.listdir(result_folder)
filenames_result.sort()

# Проверка совпадения количества файлов
if len(filenames_upload) != len(filenames_result):
    print("Внимание: количество исходных и обработанных файлов не совпадает!")

# Визуализация
for filename, filename_output in zip(filenames_upload, filenames_result):
    image_original = PIL.Image.open(os.path.join(upload_folder, filename))
    image_enhanced = PIL.Image.open(os.path.join(result_folder, filename_output))

    # Устанавливаем максимальную ширину 500 пикселей
    max_width = 500
    width_original, height_original = image_original.size
    width_enhanced, height_enhanced = image_enhanced.size

    if width_original > max_width:
        new_height = int(height_original * max_width / width_original)
        image_original = image_original.resize((max_width, new_height))
    if width_enhanced > max_width:
        new_height = int(height_enhanced * max_width / width_enhanced)
        image_enhanced = image_enhanced.resize((max_width, new_height))

    # Объединяем изображения горизонтально
    combined_image = PIL.Image.fromarray(np.hstack((np.array(image_original), np.array(image_enhanced))))

    # Отображаем результат
    display(combined_image)
    print("")

In [12]:
#@title ##**Download results** { display-mode: "form" }
#output_folder = os.path.join(upload_output_path, "final_output")
files_in_folder = os.listdir(result_folder)
zip_file = "download.zip"

if len(files_in_folder) == 1:
    file_to_download = os.path.join(result_folder, files_in_folder[0])
    files.download(file_to_download)
else:
    if os.path.exists(os.path.join(result_folder, zip_file)):
      os.remove(os.path.join(result_folder, zip_file))
    os.system(f"cd {result_folder} && zip -r -j {zip_file} * && cd ..")
    files.download(os.path.join(result_folder, zip_file))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#@title ##**Download results to google drive (optional)** { display-mode: "form" }
from google.colab import drive
drive.mount('/content/drive')

!cp "/content/Real-ESRGAN/results/download.zip" "/content/drive/MyDrive/"
